In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI()

cache_dir = LocalFileStore("./.cache/")

splitter  = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cache_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstore = FAISS.from_documents(docs, cache_embeddings)

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_reduce",
    retriever=vectorstore.as_retriever(),
)

chain.run("Describe Victory Mansions")

'Victory Mansions is a dilapidated apartment complex in George Orwell\'s novel "1984." It is a rundown, shabby building with cramped living spaces and no modern amenities. The building is constantly under surveillance by the Party with telescreens in every room. The overall atmosphere is one of oppression and decay. Winston Smith resides in this complex on the seventh floor, and the building has a roof offering a view of four towering structures including the Ministry of Truth. There is a telescreen in the living room placed in an unusual position, allowing Winston to sit in an alcove outside its range.'